In [1]:
# coding=utf-8
import numpy as np
import os
import pandas as pd
import csv
import sqlite3 as lite
from bs4 import BeautifulSoup
from selenium import webdriver
from shutil import copyfile
teenslang_appwords_csv = "teenslang_appwords.csv"
test_dict_file = "test_dict.csv"
rus_letters_numbers_path = "rus_letters_numbers.npy"

all_words_links_path = 'all_words_links.npy'
vsekidki_fullwords_csv = 'vsekidki_fullwords.csv'
vsekidki_appwords_csv = "vsekidki_appwords.csv"
vsekidki_words_page = 'vsekidki_words_npy/vsekidki_words_page'

### Functions

In [15]:
def Text_from_html_object(text):
    start = text.find("<")
    finish = text.find(">")
    text = text[:start] + text[finish + 1:]
    check_for_tag = text.find('<', start)
    while check_for_tag != -1:
            end_of_tag = text.find('>', check_for_tag)
            text = text[:check_for_tag] + text[end_of_tag + 1:]
            check_for_tag = text.find('<', check_for_tag)
    return text

def Text_from_html_arr(arr):
    answer_text = ''
    for object in arr:
        text = str(object)
        check_for_tag = text.find('<')
        while check_for_tag != -1:
            end_of_tag = text.find('>', check_for_tag)
            text = text[:check_for_tag] + text[end_of_tag + 1:]
            check_for_tag = text.find('<', check_for_tag)
        answer_text += text
    return answer_text

def get_page_count(soup):
    list_of_pages = soup.find_all('a', class_='page_number')
    try:
        last_page = list_of_pages[-2]
    except IndexError:
        return 1
    last_number = Text_from_html_object(str(last_page))
    return int(last_number)

def take_rus_letters_numbers(soup):
    links = []
    list_of_links = soup.find_all('a', class_='alph_menu')
    for link in list_of_links:
        pos = int(str(link).find("%D0%")) + 4
        name = str(str(link)[pos:pos+2])
        if name != "81":
            links.append(name)
    return links

def make_save(path):
    with open(path, 'w') as csvfile:
        writer = csv.writer(csvfile)
        writer.writerow(('name', 'definition', 'type', 'group', 'examples', 'origin', 'hashtags', 'synonyms'))

def add_to_save(dict, path):
    with open(path, 'a') as csvfile:
        writer = csv.writer(csvfile)
        for word in dict:
            type = word['type']
            group = word['group']
            examples = word['examples']
            origin = word['origin']
            hashtags = word['hashtags']
            synonyms = word['synonyms']
            writer.writerow((word['name'], word['definition'], word['type'], word['group'], 
                             word['examples'], word['origin'], word['hashtags'], word['synonyms']))
            

    
# and then rename mycsv.out to mycsv.csv if you wish.

def get_words_from_Teenslang(from_i, to_i, file):
    make_save(file)
    for j in range(from_i, to_i + 1):
        driver.get('http://teenslang.su/content/%D0%'+ rus_letters_numbers[j]) # download site to the Selenium
        html = driver.page_source # saving site source
        soup = BeautifulSoup(html, "lxml")
        page_count = get_page_count(soup)
        print('Буква алфавита № ' + str(j + 1) + ': ')
        dict = []
        for i in range(page_count):
            if i > 0:
                driver.get('http://teenslang.su/?page=' + str(i) + '&content=%D0%'+ rus_letters_numbers[j] + '*') # load sites
                html = driver.page_source # saving site source
                soup = BeautifulSoup(html, "lxml")
            print('Парсинг %d из %d' % (i + 1, page_count))
            table = soup.find('div', class_='visible-xs visible-md visible-sm visible-lg col-lg-6')
            words = table.find_all('div', class_='issueword1')

            for word_row in words:
                word_name = Text_from_html_object(str(word_row.find('span', class_='cap')))
                word_type = Text_from_html_object(str(word_row.find('span', class_='sta')))
                word_group = Text_from_html_object(str(word_row.find('span', class_='sph')))
                word_def = Text_from_html_object(str(word_row.find('span', class_='mea')))
                word_example = Text_from_html_object(str(word_row.find('span', class_='text')))
                word_story = Text_from_html_arr(word_row.find_all('div', class_='category'))
                word_hash = Text_from_html_object(str(word_row.find('span', class_='src')))
                word_syn = Text_from_html_object(str(word_row.find('span', class_='syn')))
                dict.append({
                    'num': i,
                    'name': brush(word_name),
                    'definition': word_def,
                    'type': " " if word_type == "NonNone" else word_type,
                    'group': " " if word_group == "NonNone" else word_group,
                    'examples': " " if word_example == "NonNone" else word_example,
                    'origin': " " if word_story == "NonNone" else word_story,
                    'hashtags': " " if word_hash == "NonNone" else word_hash,
                    'synonyms': " " if word_syn == "NonNone" else word_syn 
                })
        add_to_save(dict, file)
    fulldict = pd.read_csv(file)
    fulldict.drop_duplicates(keep='first')
    print("!!!!!!!!!!!ВСЕ!!!!!!!!!!")
            
        
def get_rus_letters_numbers():
    driver.get("http://teenslang.su/content/%D0%90")
    html = driver.page_source # saving site source
    soup = BeautifulSoup(html, "lxml")
    rus_letters_numbers = take_rus_letters_numbers(soup)
    return rus_letters_numbers

def brush(x):
    return x.capitalize().strip()

def print_dict_sorted_by_value(dictionary):
    for (key, value) in sorted(dictionary.items(), key=lambda x: x[1], reverse=True):
        print(key, ": ", value, sep='')
        
def load_webdriver():
    global driver
    try:
        driver = webdriver.PhantomJS()
    except:
        webdriver_path = "/home/kam/PhantomJS/phantomjs-2.1.1-linux-x86_64/bin/phantomjs"
        driver = webdriver.PhantomJS(webdriver_path)
        
def load_rus_letters_numbers():
    global rus_letters_numbers
    if os.path.isfile(rus_letters_numbers_path):
        rus_letters_numbers = np.load(rus_letters_numbers_path).tolist()
    else:
        rus_letters_numbers = get_rus_letters_numbers()
        np.save(rus_letters_numbers_path, rus_letters_numbers)
        

#### Load PhantomJS webdriver, (load from driver)/(download) numbers of Russian letters used in teenslang

In [18]:
load_webdriver()
load_rus_letters_numbers()

### test sample for letter "Й"

In [19]:
get_words_from_Teenslang(from_i=9, to_i=9, file=test_dict_file)
test_dict = pd.read_csv(test_dict_file)
test_dict.head(3)

Буква алфавита № 10: 
Парсинг 1 из 1
!!!!!!!!!!!ВСЕ!!!!!!!!!!


,name,definition,type,group,examples,origin,hashtags,synonyms
0,Йа,я.,местоимение,,Йа так устала! • Когда йа была маленькой дево...,#Падонки,,
1,Йа креведко,"я знаю олбанский язык, но не совсем понимаю, о...",,(каменты),,#Падонки,,"я чайник, прикидываюсь шлангом, шлангую, йа кр..."
2,Йааазь,"язь, название рыбы.","-я, м.",(интернет),Вот она! Вот она рыба моей мечты: здоровенный ...,#Интернет,"Из любительского видеоролика, который оказалс...","йаазь, йазь, язззь"


### All words to DataFrame

In [25]:
teenslang_df = pd.read_csv(teenslang_appwords_csv)
teenslang_df.head(3)

,name,definition,type,group,examples,hashtags,origin,synonyms,id
0,А сегодня я покакал,"Предложение, показывающие безразличие к тому ч...",словосочетание,"(Анонимы, тролли, хикки)",-Где можно обсудить проблемы кишечника?\r\n-В ...,#Социальныесети,(происхождение второго значения) раздел был до...,_,1
1,А то,да — выражения подтверждения.,союз и частица,NaN,— Ты знаешь такого чела...? — А то!!!,#Общие,NaN,да.,2
2,Абаля,вот это конфуз!.,NaN,NaN,NaN,#Общие,NaN,_,3


In [26]:
df.replace("#Общие ", np.nan)

,name,definition,type,group,examples,hashtags,origin,synonyms,id
0,А сегодня я покакал,"Предложение, показывающие безразличие к тому ч...",словосочетание,"(Анонимы, тролли, хикки)",-Где можно обсудить проблемы кишечника?\r\n-В ...,#Социальныесети,(происхождение второго значения) раздел был до...,_,1
1,А то,да — выражения подтверждения.,союз и частица,NaN,— Ты знаешь такого чела...? — А то!!!,NaN,NaN,да.,2
2,Абаля,вот это конфуз!.,NaN,NaN,NaN,NaN,NaN,_,3
3,Абанамат,восклицание раздражения или удивления.,междометие,NaN,Абанамат!!! Холодильник сломался не вовремя. ...,NaN,NaN,"Искажение, эвфемизм матероного выражения, из п...",4
4,Абапер,программист на языке ABAP/4.,"-а, м.","(айтишники\r\n, прогеры)","У абаперов, как правило, все проекты — это кос...",#Айтишники,"от англ. ABAPer, имеющего ровно то же значение.",АБАПер,5
5,Абассака,коммент к смешному креатиффу.,"-и, ж.",(каменты),NaN,#Падонки,NaN,"абасцака, ицпадстол, ицпадстул, обоссака, пацт...",6
6,абгрейд,"АПГРЕ́ЙД\r\n1.Модернизация компьютера, обновле...",Мужской род спец.,(IT),"А ты абгрейдил(апгрейдил) виндовс, до 10?",#Айтишники #Интернет,"Сфера IT, от EN слова upgrade.",АПГРЕ́ЙД.,7
7,абдул,Мужское кавказское имя.,NaN,NaN,NaN,NaN,NaN,_,8
8,абдул,Обзывание хачей.,сущ.м.р,(хачи),Абдул хач.,NaN,NaN,хас.,9
9,аберглау,Просто ляпнул - и забыл.,Существительное,(По всюду),Тоже что и Аберглау.,NaN,город Сальск.,АбайКалаУмаласаи.,10


In [17]:
for hashtags in teenslang_df['hashtags']:
    hashtags_arr = ','.join(hashtags.split())
    print(hashtags_arr)

#Социальныесети
#Общие
#Общие
#Общие
#Айтишники
#Падонки
#Айтишники,#Интернет
#Общие
#Общие
#Общие
#Общие
#Общие
#Гопники
#Общие
#Геймеры
#Геймеры
#Студенты
#Студенты
#Общие,#Ролевики,#Студенты
#Общие
#Общие
#Аниме
#Геймеры,#РПГ
#Общие
#Общие
#Общие
#Общие,#Интернет
#РПГ,#Линейка
#Интернет
#РПГ,#Линейка
#Интернет
#Геймеры,#Шутеры,#Контр-Страйк
#Интернет
#Интернет
#Интернет
#Падонки,#Интернет
#Интернет
#Автолюбители
#Автолюбители
#Общие
#Общие,#Интернет
#Автолюбители
#Геймеры,#Шутеры
#Айтишники
#Айтишники
#Айтишники
#Айтишники
#Термины
#Геймеры
#Общие
#Топонимы
#Интернет,#Падонки
#Общие
#Общие
#Флэшмоб
#Геймеры,#Общие
#Общие
#Социальныесети
#Айтишники
#Айтишники
#Общие,#Автолюбители
#Студенты
#Общие
#Автолюбители
#Термины
#Юмор
#Термины
#Общие
#Тюремныйслэнг
#ЧГКшники
#Ролевики
#Музыка
#Флэшмоб
#Общие
#Интернет
#Общие
#Общие
#Геймеры
#Общие
#Общие
#Флэшмоб
#Общие
#Общие
#Аниме
#Геймеры,#РПГ,#Стратегии
#Общие
#Геймеры
#Общие
#Геймеры
#Геймеры,#РПГ
#Геймеры
#Общие
#Общие
#Общие
#Геймеры,#

AttributeError: 'float' object has no attribute 'split'

In [5]:
NaN = teenslang_df['origin'][1]
print(NaN, type(NaN))

nan <class 'float'>


In [9]:
for row in range(len(teenslang_df)):
    hashtags = teenslang_df['hashtags'][row]
    if hashtags == '#Общие':
        teenslang_df['hashtags'][row] = NaN
    if type(hashtags) != type(NaN):
        hashtags_arr = hashtags.split()
        for hashtag in hashtags_arr:
            if hashtag == '#Общие':
                hashtags_arr.remove(hashtag)
        teenslang_df['hashtags'][row] = ','.join(hashtags_arr)
    #print(hashtags_arr)
teenslang_df.head(3)
    

/Users/lotfull/miniconda3/envs/ipykernel_py3/lib/python3.6/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # Remove the CWD from sys.path while we load stuff.


,name,definition,type,group,examples,hashtags,origin,synonyms,id
0,А сегодня я покакал,"Предложение, показывающие безразличие к тому ч...",словосочетание,"(Анонимы, тролли, хикки)",-Где можно обсудить проблемы кишечника?\r\n-В ...,#Социальныесети,(происхождение второго значения) раздел был до...,_,1
1,А то,да — выражения подтверждения.,союз и частица,NaN,— Ты знаешь такого чела...? — А то!!!,,NaN,да.,2
2,Абаля,вот это конфуз!.,NaN,NaN,NaN,,NaN,_,3


In [19]:
df = pd.read_csv(teenslang_appwords_csv)
df.head(3)

,name,definition,type,group,examples,hashtags,origin,synonyms,id
0,А сегодня я покакал,"Предложение, показывающие безразличие к тому ч...",словосочетание,"(Анонимы, тролли, хикки)",-Где можно обсудить проблемы кишечника?\r\n-В ...,#Социальныесети,(происхождение второго значения) раздел был до...,_,1
1,А то,да — выражения подтверждения.,союз и частица,NaN,— Ты знаешь такого чела...? — А то!!!,#Общие,NaN,да.,2
2,Абаля,вот это конфуз!.,NaN,NaN,NaN,#Общие,NaN,_,3


In [15]:
hashtagsSeries = df.ix[:,5]

In [ ]:
countA = 0
countB = 0
countC = 0
for i in range(len(hashtagsSeries)):
    if type(hashtagsSeries[i]) == str and hashtagsSeries[i].strip() == '#Общие':
        countA += 1
        hashtagsSeries[i] = ''
    else:
        countC += 1
print(len(hashtagsSeries), "=", countA, "+", countB, "+", countC, "=", countA+countB+countC)

/Users/lotfull/miniconda3/envs/ipykernel_py3/lib/python3.6/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys


In [42]:
len(teenslang_df)

16784

### Inserting ID column

In [44]:
teenslang_df["id"] = teenslang_df.index + 1
teenslang_df.head(1)

,name,definition,type,group,examples,hashtags,origin,synonyms,id
0,А сегодня я покакал,"Предложение, показывающие безразличие к тому ч...",словосочетание,"(Анонимы, тролли, хикки)",-Где можно обсудить проблемы кишечника?\n-В /p...,#Социальныесети,(происхождение второго значения) раздел был до...,_,1


### Save to CSV

In [27]:
teenslang_df.to_csv(teenslang_appwords_csv, index=False)

### Find by name

In [70]:
teenslang_df[teenslang_df["name"] == "Шарить"]

,name,definition,type,group,examples,hashtags,origin,synonyms,id
16937,Шарить,"хорошо разбираться в чем-либо, понимать.",несов. перех.,NaN,Он здорово шарит в компах! • Автор не шарит в...,#Общие,NaN,рубить.,16938
16938,Шарить,"Разбираться в мемах, в моде, в том, что популя...",н.ф. глагол,(Интернет),Я шарю.,#Социальныесети,NaN,_,16939


### Drop column ID by name

In [43]:
teenslang_df = teenslang_df.drop("id", 1)
teenslang_df.head(1)

,name,definition,type,group,examples,hashtags,origin,synonyms
0,А сегодня я покакал,"Предложение, показывающие безразличие к тому ч...",словосочетание,"(Анонимы, тролли, хикки)",-Где можно обсудить проблемы кишечника?\n-В /p...,#Социальныесети,(происхождение второго значения) раздел был до...,_


### Drop duplicates

In [88]:
teenslang_df.drop_duplicates(inplace=True, keep='first')
len(teenslang_df)

16751

### Repeating with Vsekidki_dict

In [7]:
vsekidki_df = pd.read_csv(vsekidki_appwords_csv)
vsekidki_df.head(1)

,name,definition,type,group,examples,origin,hashtags,synonyms,id
0,#Mannequinchallenge,"Манекен Челлендж - вирусный тренд, в котором в...",NaN,NaN,NaN,"Самый крутой челлендж, по-моему мнению, на дан...",NaN,_,1


In [8]:
list(vsekidki_df)

['name',
 'definition',
 'type',
 'group',
 'examples',
 'origin',
 'hashtags',
 'synonyms',
 'id']

In [9]:
columnsTitles=['name',
 'definition',
 'type',
 'group',
 'examples',
 'hashtags',
 'origin',
 'synonyms',
 'id']
vsekidki_df=vsekidki_df.reindex(columns=columnsTitles)
vsekidki_df.head(1)

,name,definition,type,group,examples,hashtags,origin,synonyms,id
0,#Mannequinchallenge,"Манекен Челлендж - вирусный тренд, в котором в...",NaN,NaN,NaN,NaN,"Самый крутой челлендж, по-моему мнению, на дан...",_,1


In [ ]:
vsekidki_df = vsekidki_df.drop("id", 1)
vsekidki_df.drop_duplicates(inplace=True, keep='first')
vsekidki_df["id"] = vsekidki_df.index + 1

In [10]:
vsekidki_df.to_csv(vsekidki_appwords_csv, index=False)
len(vsekidki_df)

1307